In [1]:
import re
import requests
import time
from Bio import Entrez
import nltk
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from gensim import corpora
from gensim.models import LdaModel
from collections import Counter
from textblob import TextBlob
from wordcloud import WordCloud

In [3]:
import requests
from Bio import Entrez

base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
retmax = 10
api_key = '2a712e0d47f3436ef738ec764ade7b1bee09'

# Define the range of years you want to search
start_year = 2010
end_year = 2020

# Collect PubMed IDs by year
all_ids_by_year = {}

for year in range(start_year, end_year + 1):
    ids_for_year = []
    for retstart in range(0, 10, retmax):
        search_term = f"secondary metabolite AND plants AND {year}[Publication Date]"
        search_url = f"{base_url}esearch.fcgi?db=pubmed&term={search_term}&retmax={retmax}&retstart={retstart}&retmode=json"

        if api_key:
            search_url += f"&api_key={api_key}"

        search_response = requests.get(search_url)
        search_data = search_response.json()

        if "ERROR" in search_data["esearchresult"]:
            print(search_data["esearchresult"]["ERROR"])
            break

        current_ids = search_data["esearchresult"]["idlist"]
        ids_for_year.extend(current_ids)

        if len(current_ids) < retmax:
            break

    all_ids_by_year[year] = ids_for_year

# Fetch abstracts for the collected PubMed IDs
def fetch_abstracts_by_year(pubmed_ids_by_year):
    Entrez.email = "your_email@example.com"  # Provide your email address for NCBI access
    
    details_by_year = {}
    
    for year, pubmed_ids in pubmed_ids_by_year.items():
        handle = Entrez.efetch(db="pubmed", id=','.join(map(str, pubmed_ids)), retmode="xml")
        records = Entrez.read(handle)
        
        abstracts = {}
        
        for record in records['PubmedArticle']:
            pubmed_id = record['PubmedData']['ArticleIdList'][0]
            
            abstract_text = ''
            if 'Abstract' in record['MedlineCitation']['Article']:
                abstract_text = record['MedlineCitation']['Article']['Abstract']['AbstractText']
            
            abstracts[pubmed_id] = {'Abstract': abstract_text}
        
        details_by_year[year] = abstracts
    
    return details_by_year

# Use the collected PubMed IDs to fetch abstracts
abstracts_by_year = fetch_abstracts_by_year(all_ids_by_year)

# Print the results
for year, abstracts in abstracts_by_year.items():
    print(f"Year: {year}")
    for pubmed_id, data in abstracts.items():
        print(f"PubMed ID: {pubmed_id}")
        print(f"Abstract: {data.get('Abstract', 'N/A')}")
        print("\n")

Year: 2010
PubMed ID: 23059762
Abstract: ['Plumbagin is one of the simplest plant secondary metabolite of three major phylogenic families viz. Plumbaginaceae, Droseraceae, and Ebenceae, and exhibits highly potent biological activities, including antioxidant, antiinflammatory, anticancer, antibacterial, and antifungal activities. Recent investigations indicate that these activities arise mainly out of its ability to undergo redox cycling, generating reactive oxygen species and chelating trace metals in biological system. The compound is endowed with a property to inhibit the drug efflux mechanism in drug-resistant bacteria, thereby allowing intracellular accumulation of the potent drug molecules. An interesting bioactivity exhibited by this compound is the elimination of stringent, conjugative, multidrug-resistant plasmids from several bacterial strains including opportunistic bacteria, such as Acinetobacter baumannii. Moreover, plumbagin effectively induces apoptosis and causes cell cy